In [1]:
import sys

sys.path.append("/home/xmh/DeepCTR-Torch")

# -*- coding: utf-8 -*-
import pandas as pd
import torch
from sklearn.metrics import log_loss, roc_auc_score,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names
from deepctr_torch.models import *

import numpy as np

In [2]:
data_prefix = "/data/avazu-ctr-prediction/"
df = pd.read_csv(data_prefix + "train-mini")
# df = pd.read_csv(data_prefix + "train")

In [3]:
data = df
sparse_features = ['id', 'C1', 'banner_pos', 'site_id', 'site_domain',
                   'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
                   'device_ip', 'device_model', 'device_type', 'device_conn_type', ] \
                  + ['C' + str(i) for i in range(14, 22)]
dense_features = []

target = ['click']

# 1.Label Encoding for sparse features,and do simple Transformation for dense features

mms = MinMaxScaler(feature_range=(0, 1))
if dense_features != []:
    data[dense_features] = mms.fit_transform(data[dense_features])

for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])

# 2.count #unique features for each sparse field,and record dense feature field name

fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].nunique(), embedding_dim=4)
                          for i, feat in enumerate(sparse_features)] + [DenseFeat(feat, 1,)
                                                                        for feat in dense_features]

dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

# 3.generate input data for modelf

train, test = train_test_split(data, test_size=0.1)
train_model_input = {name: train[name] for name in feature_names}
test_model_input = {name: test[name] for name in feature_names}

In [5]:
# 4.Define Model,train,predict and evaluate
import argparse

parser = argparse.ArgumentParser()
parser.add_argument("--model", choices=('deepfm',
                                        'wdl', 'dcn'),
                    default='deepfm')

parser.add_argument("--dense-opt", choices=('adam',
                                            'sgd', 'adagrad', 'adadelta'), default='adam')
parser.add_argument("--sparse-opt", choices=('adam',
                                             'sgd', 'adagrad', 'adadelta', 'rrms'), default=None)
# 混合有问题


parser.add_argument("--dense-lr", type=float, default=0.1)
parser.add_argument("--sparse-lr", type=float, default=0.1)


args = parser.parse_args(args=[])


model_name = args.model
optimizer_dense = args.dense_opt
optimizer_sparse = args.sparse_opt
optimizer_dense_lr = args.dense_lr
optimizer_sparse_lr = args.sparse_lr

device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

if model_name == "deepfm":
    model = DeepFM(linear_feature_columns, dnn_feature_columns,
                   task='binary', device=device)
# elif model_name == "din":
#     model = DIN(linear_feature_columns, dnn_feature_columns,
#                 task='binary', device=device)
elif model_name == "wdl":
    model = WDL(linear_feature_columns, dnn_feature_columns,
                task='binary', device=device)
elif model_name == "dcn":
    model = DCN(linear_feature_columns, dnn_feature_columns,
                task='binary', device=device)

model.compile(optimizer=optimizer_dense, loss="binary_crossentropy",
              metrics=['binary_crossentropy', 'acc', 'AUC'],
              optimizer_sparse=optimizer_sparse,
              optimizer_dense_lr=optimizer_dense_lr,
              optimizer_sparse_lr=optimizer_sparse_lr, )
history = model.fit(train_model_input, train[target].values,
                    batch_size=128, epochs=1, verbose=1, validation_split=0.1, model_name=model_name,
                   verbose_steps= 100)
pred_ans = model.predict(test_model_input, batch_size=128)
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test Accuracy", round(accuracy_score(
    test[target].values, pred_ans > 0.5), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))

2it [00:00, 16.85it/s]

cpu
Train on 80999 samples, validate on 9000 samples, 633 steps per epoch


104it [00:04, 15.24it/s]

[Iter99] - loss:  0.4688 - binary_crossentropy:  0.4688 - acc:  0.8261 - auc:  0.6605 - val_binary_crossentropy:  0.4222 - val_acc:  0.8274 - val_auc:  0.7075


203it [00:09, 14.22it/s]

...

[Iter499] - loss:  0.4200 - binary_crossentropy:  0.4200 - acc:  0.8276 - auc:  0.7271 - val_binary_crossentropy:  0.4006 - val_acc:  0.8279 - val_auc:  0.7616


604it [00:31, 14.20it/s]

[Iter599] - loss:  0.4173 - binary_crossentropy:  0.4173 - acc:  0.8273 - auc:  0.7322 - val_binary_crossentropy:  0.3953 - val_acc:  0.8282 - val_auc:  0.7700


633it [00:33, 18.99it/s]


Epoch 1/1
33s - loss:  0.4165 - binary_crossentropy:  0.4165 - acc:  0.8272 - auc:  0.7340 - val_binary_crossentropy:  0.3944 - val_acc:  0.8297 - val_auc:  0.7717
test LogLoss 0.3914
test Accuracy 0.8314
test AUC 0.7646
